# Excercises 
# 1. Tune the network
Run the experiment below, explore the different parameters (see suggestions below) and study the result with tensorboard. 
Make a single page (1 a4) report of your findings. Use your visualisation skills to communicate your most important findings.

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
%tensorboard --logdir modellogs/

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Alex\AppData\Local\Programs\Python\Python312\Scripts\tensorboard.exe\__main__.py", line 2, in <module>
  File "C:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorboard\main.py", line 27, in <module>
    from tensorboard import default
  File "C:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorboard\default.py", line 30, in <module>
    import pkg_resources
ModuleNotFoundError: No module named 'pkg_resources'

In [ ]:
import sys
print(sys.executable)

In [ ]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
from torch import nn
from tomlserializer import TOMLSerializer

We will be using `tomlserializer` to easily keep track of our experiments, and to easily save the different things we did during our experiments.
It can export things like settings and models to a simple `toml` file, which can be easily shared, checked and modified.

First, we need the data. 

In [ ]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

We need a way to determine how well our model is performing. We will use accuracy as a metric.

In [ ]:
accuracy = metrics.Accuracy()

You can set up a single experiment.

- We will show the model batches of 64 images, 
- and for every epoch we will show the model 100 batches (trainsteps=100).
- then, we will test how well the model is doing on unseen data (teststeps=100).
- we will report our results during training to tensorboard, and report all configuration to a toml file.
- we will log the results into a directory called "modellogs", but you could change this to whatever you want.

In [ ]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)


We will use a very basic model: a model with three linear layers.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(
    num_classes=10, units1=256, units2=256)

I developped the `tomlserializer` package, it is a useful tool to save configs, models and settings as a tomlfile; that way it is easy to track what you changed during your experiments.

This package will 1. check if there is a `__dict__` attribute available, and if so, it will use that to extract the parameters that do not start with an underscore, like this:

In [ ]:
{k: v for k, v in model.__dict__.items() if not k.startswith("_")}

This means that if you want to add more parameters to the `.toml` file, eg `units3`, you can add them to the class like this:

```python
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int, units3: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.units3 = units3  # <-- add this line
```

And then it will be added to the `.toml` file. Check the result for yourself by using the `.save()` method of the `TomlSerializer` class like this:

In [ ]:
tomlserializer = TOMLSerializer()
tomlserializer.save(settings, "settings.toml")
tomlserializer.save(model, "model.toml")

Check the `settings.toml` and `model.toml` files to see what is in there.

You can use the `Trainer` class from my `mltrainer` module to train your model. It has the TOMLserializer integrated, so it will automatically save the settings and model to a toml file if you have added `TOML` as a reporttype in the settings.

In [ ]:
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)
trainer.loop()

Now, check in the modellogs directory the results of your experiment.

We can now loop this with a naive approach, called a grid-search (why do you think i call it naive?).

In [ ]:
units = [256, 128, 64]
for unit1 in units:
    for unit2 in units:
        print(f"Units: {unit1}, {unit2}")

Of course, this might not be the best way to search for a model; some configurations will be better than others (can you predict up front what will be the best configuration?).

So, feel free to improve upon the gridsearch by adding your own logic.

In [ ]:
import torch

# --- Definieer hier je 10 combinaties ---
experiments = [
    {"units1": 256, "units2": 256, "epochs": 5,  "batchsize": 32, "lr": 1e-3, "optimizer": "SGD"},
    {"units1": 256, "units2": 128, "epochs": 5,  "batchsize": 64, "lr": 1e-4, "optimizer": "Adam"},
    {"units1": 128, "units2": 128, "epochs": 10, "batchsize": 32, "lr": 1e-3, "optimizer": "Adam"},
    {"units1": 128, "units2": 64,  "epochs": 10, "batchsize": 64, "lr": 1e-4, "optimizer": "SGD"},
    {"units1": 64,  "units2": 64,  "epochs": 5,  "batchsize": 32, "lr": 1e-3, "optimizer": "SGD"},
    {"units1": 64,  "units2": 128, "epochs": 10, "batchsize": 64, "lr": 1e-4, "optimizer": "Adam"},
    {"units1": 256, "units2": 64,  "epochs": 5,  "batchsize": 64, "lr": 1e-4, "optimizer": "Adam"},
    {"units1": 128, "units2": 256, "epochs": 10, "batchsize": 32, "lr": 1e-3, "optimizer": "SGD"},
    {"units1": 64,  "units2": 256, "epochs": 5,  "batchsize": 64, "lr": 1e-4, "optimizer": "Adam"},
    {"units1": 256, "units2": 128, "epochs": 10, "batchsize": 32, "lr": 1e-3, "optimizer": "Adam"},
]

# CrossEntropyLoss voor classificatie
for exp in experiments:
    model = NeuralNetwork(num_classes=10, units1=exp["units1"], units2=exp["units2"])

    # TrainerSettings, inclusief learning rate
    settings = TrainerSettings(
        epochs=exp["epochs"],
        metrics=[accuracy],
        logdir=f"modellogs/u1_{exp['units1']}_u2_{exp['units2']}_ep_{exp['epochs']}_bs_{exp['batchsize']}_lr_{exp['lr']}_{exp['optimizer']}",
        train_steps=len(train),
        valid_steps=len(valid),
        reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
        learningrate=exp["lr"]  # learning rate via settings
    )

    # Optimizerklasse doorgeven, géén instantie
    if exp["optimizer"] == "SGD":
        opt_class = torch.optim.SGD
    else:
        opt_class = torch.optim.Adam

    # Trainer
    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=opt_class,  # klasse, geen instantie
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau
    )

    trainer.loop()

Because we have set the ReportType to TOML, you will find in every log dir a model.toml and settings.toml file.

Run the experiment, and study the result with tensorboard. 

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

- in the terminal, `cd` to the location of the repository
- activate the python environment for the shell. Note how the correct environment is being activated.
- run `tensorboard --logdir=modellogs` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`